In [31]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.autograd import Variable
import pandas as pd
import numpy as np
torch.manual_seed(777)
from gensim.models import Word2Vec
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
from torch.utils.data import DataLoader,Dataset
from sklearn.metrics import f1_score,confusion_matrix,accuracy_score,precision_score,recall_score
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from pprint import pprint

from gensim.models import KeyedVectors
%matplotlib inline

In [7]:
import pickle

with open("./pickle_data/token_data.pickle", "rb") as f:
    token_data = pickle.load(f)

In [8]:
token_data.keys()

dict_keys(['comment', 'target', 'comment_ix', 'word2ix', 'ix2word', 'max_seq_length'])

In [9]:
comment=token_data['comment']
target=token_data['target']
comment_idx=token_data['comment_ix']
word2idx=token_data['word2ix']
idx2word=token_data['ix2word']

In [37]:
li1=[]
question_list=[]
for i in comment:
    for k in i:
        li1.append(word2idx[k])
    
    question_list.append(li1)
    li1=[]

In [38]:
model_word2vec=Word2Vec.load('./word2vec_model/word2vec_100dim_1115.model')

In [39]:
model_word2vec.vector_size

100

In [40]:
embedding_matrix = []

for word in word2idx.keys():
    try:
        embedding_matrix.append(model_word2vec[word])
    except:
        embedding_matrix.append(np.zeros(100))

print(len(word2idx))
print(len(embedding_matrix))

embedding_matrix = torch.Tensor(embedding_matrix)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


17786
17786


In [131]:
class Conv_net_1d(nn.Module):
    def __init__(self,out_channels,num_class,input_size,embedding_matrix,filter_list):
        
        super(Conv_net_1d,self).__init__()
        
        self.embed=nn.Embedding.from_pretrained(embeddings=embedding_matrix)
        self.filter_1= nn.Sequential(
                            nn.Conv1d(in_channels=1,stride=1,padding=0,kernel_size=filter_list[0]*input_size,out_channels=out_channels),
                            nn.BatchNorm1d(out_channels),
                            nn.ReLU(),
                        )
        self.filter_2= nn.Sequential(
                            nn.Conv1d(in_channels=1,stride=1,padding=0,kernel_size=filter_list[1]*input_size,out_channels=out_channels),
                            nn.BatchNorm1d(out_channels),
                            nn.ReLU(),
                        )
        self.filter_3= nn.Sequential(
                            nn.Conv1d(in_channels=1,stride=1,padding=0,kernel_size=filter_list[2]*input_size,out_channels=out_channels),
                            nn.BatchNorm1d(out_channels),
                            nn.ReLU(),
                        )
        
        self.dropout=nn.Dropout(0.7)
        self.fc=nn.Linear(out_channels*len(filter_list),num_class)
        
    def forward(self,x):
        x=self.embed(x)
        
        x_cat=x.reshape(batch_size, 1, -1)
        
        out=[self.filter_1(x_cat),self.filter_2(x_cat),self.filter_3(x_cat)]
        
        out=[F.max_pool1d(conv,(conv.size(2),)).squeeze(2) for conv in out]
        
        out=torch.cat(out,1)
        
        out=self.dropout(out)
        
        out=self.fc(out)
        
        out=F.softmax(out,dim=1)
        
        return out
            
    def predict(self,x,test_batch_size):
    
        x=self.embed(x)
        x_cat=x.reshape(test_batch_size, 1, -1)
        
        out=[self.filter_1(x_cat),self.filter_2(x_cat),self.filter_3(x_cat)]
        out=[F.max_pool1d(conv,(conv.size(2),)).squeeze(2) for conv in out]
        
        out = torch.cat(out, 1)
        out = self.fc(out)
        out = F.softmax(out, dim=1)
        
        
        
        return out
    

In [66]:
X_train,X_test,y_train,y_test=train_test_split(question_list,target,test_size=0.34,random_state=777,stratify=target)

In [67]:
class Dataset(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)

train_data = Dataset(np.array(X_train), y_train)
test_data = Dataset(np.array(X_test), y_test)

In [68]:
filter_list=[3,4,5]

In [69]:
filter_sizes=[3,4,5]

In [158]:
def pad_sequence(batch,batch_size):
        X_batch, y_batch = zip(*batch)
        max_seq_length = max([len(x) for x in X_batch])
        if max_seq_length < max(filter_list):
            max_seq_length = max(filter_list)

        res = []
        for seq in X_batch:
            if len(seq) < max_seq_length:
                pad_seq = torch.LongTensor(seq + [0]*(max_seq_length-len(seq)))
                res.append(pad_seq)
            else:
                res.append(torch.LongTensor(seq))
        return torch.cat(res).reshape(batch_size, max_seq_length), torch.LongTensor(y_batch)

def pad_sequence_t(batch):
    X_batch, y_batch = zip(*batch)
    max_seq_length = max([len(x) for x in X_batch])
    if max_seq_length < max(filter_list):
        max_seq_length = max(filter_list)

    res = []
    for seq in X_batch:
        if len(seq) < max_seq_length:
            pad_seq = torch.LongTensor(seq + [0]*(max_seq_length-len(seq)))
            res.append(pad_seq)
        else:
            res.append(torch.LongTensor(seq))
    return torch.cat(res).reshape(test_batch_size, max_seq_length), torch.LongTensor(y_batch)

In [133]:
model=Conv_net_1d(embedding_matrix=embedding_matrix,filter_list=[3,4,5],num_class=2,out_channels=100,input_size=100).to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(),lr=0.01) # use L2-Norm

In [134]:
batch_size=166
test_batch_size=1

In [135]:
y_batch

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1])

In [136]:
X_batch

tensor([[17550,     0,     0,     0,     0],
        [ 4839,  6845, 15632,  8190,     0],
        [12348,  1217,     0,     0,     0],
        [ 8845,  3071,     0,     0,     0],
        [ 8513,     0,     0,     0,     0],
        [16908,     0,     0,     0,     0],
        [ 3309,     0,     0,     0,     0],
        [15301, 16344,     0,     0,     0],
        [11943,  8944,  4087,  5539,     0],
        [12813,     0,     0,     0,     0],
        [ 2662,   997,     0,     0,     0],
        [ 4065, 16853,     0,     0,     0],
        [10161,  1384, 14386,     0,     0],
        [ 8848, 10959,     0,     0,     0],
        [ 2887,     0,     0,     0,     0],
        [  324, 12009,  6783,  5876,     0],
        [15745, 16728,     0,     0,     0],
        [ 1217,     0,     0,     0,     0],
        [ 4092,     0,     0,     0,     0],
        [ 2066,     0,     0,     0,     0],
        [ 2221,     0,     0,     0,     0],
        [ 1902,  6217,     0,     0,     0],
        [ 

In [143]:
test_batch_size=1

In [160]:
train_loader=DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True, collate_fn=pad_sequence)
test_loader=DataLoader(dataset=test_data, batch_size=1, shuffle=False)

In [154]:
train_data.X_data

array([ list([7659, 4087, 11355, 17251, 3445, 15251, 287, 7178, 10757, 8278, 4991, 9479, 6344, 14292, 8163, 9877, 813, 11526, 7000, 9672, 8163, 8635, 6344, 10530, 4087, 251, 2659, 5243, 4087, 8173, 6188, 2662, 8278, 15251, 7721, 1945, 8173]),
       list([6383, 5234, 9432, 14672, 5111]), list([9985, 5605, 7024]),
       ..., list([16605, 8190, 4261]), list([15251, 14490]),
       list([11957, 5527, 15277])], dtype=object)

In [153]:
test_data.X_data

array([list([17550]), list([4839, 6845, 15632, 8190]), list([12348, 1217]),
       ..., list([14490]), list([2662, 1185, 8635]), list([6235])], dtype=object)

In [156]:
for seq in X_batch:
    print(seq)

tensor([6235])


[tensor([6235])]

In [172]:
print(word2idx.get('<PAD>'))

0


In [178]:
for X_batch,y_batch in test_loader:

    print(*(X_batch)+[word2idx.get('<PAD>')],y_batch)

tensor([17550]) 0 tensor([0])
tensor([4839]) tensor([6845]) tensor([15632]) tensor([8190]) 0 tensor([0])
tensor([12348]) tensor([1217]) 0 tensor([0])
tensor([8845]) tensor([3071]) 0 tensor([0])
tensor([8513]) 0 tensor([0])
tensor([16908]) 0 tensor([0])
tensor([3309]) 0 tensor([0])
tensor([15301]) tensor([16344]) 0 tensor([0])
tensor([11943]) tensor([8944]) tensor([4087]) tensor([5539]) 0 tensor([0])
tensor([12813]) 0 tensor([1])
tensor([2662]) tensor([997]) 0 tensor([0])
tensor([4065]) tensor([16853]) 0 tensor([0])
tensor([10161]) tensor([1384]) tensor([14386]) 0 tensor([0])
tensor([8848]) tensor([10959]) 0 tensor([0])
tensor([2887]) 0 tensor([1])
tensor([324]) tensor([12009]) tensor([6783]) tensor([5876]) 0 tensor([0])
tensor([15745]) tensor([16728]) 0 tensor([0])
tensor([1217]) 0 tensor([1])
tensor([4092]) 0 tensor([0])
tensor([2066]) 0 tensor([0])
tensor([2221]) 0 tensor([0])
tensor([1902]) tensor([6217]) 0 tensor([1])
tensor([9672]) tensor([5690]) 0 tensor([1])
tensor([13265]) tens

tensor([17389]) tensor([16908]) tensor([17550]) 0 tensor([0])
tensor([3043]) tensor([8561]) 0 tensor([0])
tensor([17308]) 0 tensor([0])
tensor([7566]) tensor([17276]) tensor([8116]) 0 tensor([0])
tensor([12647]) tensor([15706]) 0 tensor([0])
tensor([14668]) tensor([8185]) tensor([2662]) 0 tensor([0])
tensor([17258]) tensor([11772]) tensor([1606]) 0 tensor([0])
tensor([13101]) tensor([13905]) tensor([11121]) 0 tensor([0])
tensor([7716]) 0 tensor([0])
tensor([16620]) tensor([16261]) tensor([16024]) tensor([11061]) tensor([1028]) tensor([13903]) tensor([10938]) 0 tensor([0])
tensor([15572]) tensor([13999]) 0 tensor([1])
tensor([17489]) tensor([2765]) 0 tensor([1])
tensor([16028]) tensor([11529]) tensor([9153]) tensor([8190]) tensor([10295]) tensor([6493]) 0 tensor([0])
tensor([12765]) tensor([4588]) tensor([1217]) 0 tensor([0])
tensor([16712]) tensor([16778]) 0 tensor([1])
tensor([4087]) tensor([14280]) tensor([7306]) 0 tensor([0])
tensor([11773]) tensor([2039]) tensor([15550]) 0 tensor([

tensor([9019]) tensor([2400]) tensor([10178]) 0 tensor([0])
tensor([8278]) tensor([9844]) tensor([5541]) tensor([8432]) 0 tensor([0])
tensor([2104]) tensor([15007]) 0 tensor([0])
tensor([17489]) tensor([5127]) tensor([13233]) tensor([8787]) 0 tensor([0])
tensor([6707]) tensor([11957]) tensor([8702]) tensor([8068]) 0 tensor([0])
tensor([1132]) tensor([5032]) tensor([2758]) 0 tensor([1])
tensor([9844]) tensor([9926]) tensor([13318]) tensor([8185]) 0 tensor([0])
tensor([12143]) tensor([12752]) tensor([3695]) 0 tensor([0])
tensor([16605]) tensor([1043]) tensor([17299]) tensor([11899]) tensor([7932]) tensor([9737]) tensor([16758]) tensor([1043]) tensor([3850]) 0 tensor([0])
tensor([589]) tensor([8015]) tensor([4087]) tensor([1810]) tensor([5157]) 0 tensor([0])
tensor([589]) 0 tensor([1])
tensor([5529]) tensor([6970]) tensor([4555]) 0 tensor([1])
tensor([6635]) tensor([4717]) tensor([15625]) tensor([9284]) tensor([1255]) tensor([6920]) 0 tensor([0])
tensor([7566]) tensor([6354]) tensor([1217

tensor([9991]) tensor([4551]) tensor([13155]) tensor([16040]) 0 tensor([1])
tensor([589]) tensor([6399]) 0 tensor([1])
tensor([7725]) tensor([9416]) tensor([5774]) tensor([13332]) tensor([10440]) tensor([14503]) tensor([2662]) tensor([1615]) tensor([4463]) 0 tensor([0])
tensor([15210]) tensor([13999]) 0 tensor([0])
tensor([8980]) tensor([1043]) 0 tensor([1])
tensor([6946]) tensor([13705]) tensor([16908]) 0 tensor([0])
tensor([5834]) tensor([12798]) tensor([3329]) tensor([13770]) tensor([17044]) 0 tensor([0])
tensor([3043]) tensor([2803]) tensor([8063]) tensor([14461]) 0 tensor([1])
tensor([717]) tensor([9672]) tensor([12664]) 0 tensor([0])
tensor([17439]) tensor([14798]) 0 tensor([1])
tensor([16605]) tensor([14153]) tensor([11456]) tensor([8015]) 0 tensor([0])
tensor([244]) tensor([1140]) 0 tensor([1])
tensor([375]) tensor([10701]) tensor([7584]) 0 tensor([1])
tensor([9801]) 0 tensor([1])
tensor([2758]) tensor([2699]) tensor([11572]) 0 tensor([1])
tensor([15036]) tensor([51]) tensor([5

tensor([10801]) tensor([11557]) tensor([16846]) tensor([7224]) tensor([14855]) tensor([11557]) tensor([4183]) 0 tensor([0])
tensor([7433]) tensor([5834]) tensor([17439]) 0 tensor([0])
tensor([2141]) tensor([4787]) 0 tensor([0])
tensor([8338]) tensor([4601]) tensor([16621]) tensor([14153]) tensor([14645]) tensor([16531]) 0 tensor([0])
tensor([13220]) tensor([17118]) tensor([13319]) tensor([6987]) tensor([4839]) tensor([14815]) tensor([5592]) 0 tensor([0])
tensor([4087]) tensor([11304]) tensor([10802]) 0 tensor([0])
tensor([12246]) 0 tensor([1])
tensor([4152]) tensor([4244]) tensor([997]) 0 tensor([0])
tensor([15180]) tensor([5075]) tensor([15331]) 0 tensor([0])
tensor([2531]) 0 tensor([0])
tensor([15703]) tensor([5279]) tensor([1639]) tensor([9354]) tensor([9689]) tensor([10344]) tensor([11998]) tensor([7456]) 0 tensor([0])
tensor([3397]) tensor([5009]) tensor([8569]) 0 tensor([0])
tensor([16605]) tensor([10342]) tensor([8015]) 0 tensor([0])
tensor([15621]) tensor([8320]) tensor([16105]

...

tensor([1132]) tensor([5592]) tensor([15647]) tensor([8351]) tensor([7519]) tensor([8432]) 0 tensor([1])
tensor([13367]) tensor([15517]) tensor([14093]) 0 tensor([0])
tensor([5834]) tensor([15443]) tensor([13931]) tensor([5279]) tensor([2319]) tensor([1026]) 0 tensor([0])
tensor([13937]) tensor([15232]) 0 tensor([0])
tensor([16376]) tensor([12346]) tensor([1893]) 0 tensor([0])
tensor([16675]) tensor([10223]) tensor([7128]) tensor([7276]) tensor([13601]) tensor([16345]) tensor([10349]) tensor([7011]) tensor([4618]) 0 tensor([0])
tensor([13757]) tensor([9947]) tensor([12373]) 0 tensor([0])
tensor([6702]) tensor([375]) 0 tensor([0])
tensor([153]) tensor([3808]) tensor([8432]) 0 tensor([0])
tensor([11405]) tensor([9548]) tensor([2337]) 0 tensor([0])
tensor([2100]) tensor([11576]) tensor([12046]) 0 tensor([0])
tensor([1109]) tensor([13999]) 0 tensor([0])
tensor([6756]) 0 tensor([0])
tensor([17521]) tensor([14824]) 0 tensor([0])
tensor([13999]) 0 tensor([0])
tensor([3796]) tensor([14299]) te

tensor([1440]) tensor([4840]) tensor([4911]) 0 tensor([0])
tensor([10180]) 0 tensor([1])
tensor([13963]) tensor([2580]) tensor([8173]) 0 tensor([0])
tensor([14347]) tensor([6188]) tensor([6166]) tensor([12720]) 0 tensor([0])
tensor([8853]) tensor([13922]) 0 tensor([0])
tensor([8015]) tensor([1810]) tensor([5157]) 0 tensor([0])
tensor([8513]) tensor([16105]) 0 tensor([1])
tensor([1987]) tensor([8015]) tensor([2489]) tensor([16853]) 0 tensor([0])
tensor([2375]) tensor([109]) 0 tensor([0])
tensor([6207]) tensor([15572]) tensor([11999]) 0 tensor([0])
tensor([13866]) tensor([10349]) tensor([2320]) 0 tensor([1])
tensor([12328]) tensor([4456]) tensor([15904]) tensor([5234]) tensor([874]) tensor([2213]) 0 tensor([0])
tensor([10906]) tensor([13793]) tensor([12722]) 0 tensor([0])
tensor([14915]) tensor([1133]) tensor([2538]) tensor([13715]) tensor([3221]) tensor([16799]) tensor([11239]) tensor([13144]) tensor([14690]) tensor([7758]) tensor([967]) tensor([7928]) tensor([3616]) tensor([2433]) tens

tensor([9985]) tensor([5756]) tensor([2215]) 0 tensor([0])
tensor([9905]) tensor([4619]) tensor([1555]) 0 tensor([0])
tensor([1457]) tensor([324]) tensor([12009]) 0 tensor([0])
tensor([15865]) 0 tensor([0])
tensor([2476]) tensor([8540]) tensor([13352]) 0 tensor([0])
tensor([589]) 0 tensor([1])
tensor([14321]) 0 tensor([0])
tensor([3236]) 0 tensor([0])
tensor([1120]) tensor([16908]) 0 tensor([0])
tensor([14102]) tensor([17672]) tensor([3352]) 0 tensor([0])
tensor([3243]) 0 tensor([1])
tensor([8692]) tensor([12841]) 0 tensor([0])
tensor([7063]) tensor([8320]) 0 tensor([0])
tensor([12246]) 0 tensor([1])
tensor([5314]) tensor([3681]) tensor([6335]) 0 tensor([0])
tensor([712]) tensor([15909]) tensor([17165]) 0 tensor([0])
tensor([11552]) tensor([5614]) 0 tensor([0])
tensor([9887]) tensor([14490]) 0 tensor([0])
tensor([11187]) tensor([3224]) tensor([15984]) tensor([17531]) 0 tensor([0])
tensor([15067]) tensor([11032]) tensor([15760]) tensor([8432]) 0 tensor([0])
tensor([8163]) tensor([6307])

tensor([16987]) tensor([4519]) tensor([7716]) 0 tensor([1])
tensor([4637]) 0 tensor([0])
tensor([10478]) tensor([6265]) 0 tensor([0])
tensor([8988]) tensor([14271]) 0 tensor([0])
tensor([15067]) tensor([3639]) tensor([17187]) 0 tensor([0])
tensor([15905]) tensor([8205]) 0 tensor([0])
tensor([589]) 0 tensor([1])
tensor([10269]) tensor([6609]) tensor([11330]) 0 tensor([0])
tensor([13411]) tensor([2731]) tensor([16389]) 0 tensor([1])
tensor([12149]) tensor([14672]) tensor([6017]) tensor([6264]) tensor([15538]) 0 tensor([0])
tensor([9685]) tensor([12014]) tensor([10851]) tensor([4615]) tensor([17163]) 0 tensor([0])
tensor([589]) 0 tensor([0])
tensor([12601]) tensor([8320]) 0 tensor([0])
tensor([14386]) 0 tensor([1])
tensor([17462]) tensor([695]) tensor([5898]) tensor([11773]) 0 tensor([0])
tensor([9281]) tensor([16605]) tensor([17212]) 0 tensor([0])
tensor([12974]) tensor([6759]) tensor([7510]) tensor([5637]) tensor([13972]) tensor([8936]) 0 tensor([0])
tensor([17279]) tensor([9334]) tenso

tensor([7659]) tensor([4087]) tensor([11355]) tensor([17251]) tensor([3445]) tensor([15251]) tensor([287]) tensor([7178]) tensor([10757]) tensor([8278]) tensor([4991]) tensor([9479]) tensor([6344]) tensor([14292]) tensor([8163]) tensor([9877]) tensor([813]) tensor([11526]) tensor([7000]) tensor([9672]) tensor([8163]) tensor([8635]) tensor([6344]) tensor([10530]) tensor([4087]) tensor([251]) tensor([2659]) tensor([5243]) tensor([4087]) tensor([8173]) tensor([6188]) tensor([2662]) tensor([8278]) tensor([15251]) tensor([7721]) tensor([1945]) tensor([8173]) 0 tensor([0])
tensor([14065]) 0 tensor([0])
tensor([1457]) tensor([2671]) tensor([5592]) tensor([16468]) 0 tensor([0])
tensor([16908]) 0 tensor([1])
tensor([12193]) tensor([8393]) tensor([16533]) tensor([2398]) tensor([5870]) 0 tensor([0])
tensor([6143]) tensor([4143]) 0 tensor([0])
tensor([6532]) 0 tensor([0])
tensor([16093]) tensor([2965]) tensor([767]) 0 tensor([0])
tensor([15909]) tensor([16101]) tensor([17206]) 0 tensor([1])
tensor

In [147]:
%%time
for epoch in range(10):
    train_correct = 0
    train_count = 0
    train_loss = 0
    val_correct = 0
    val_count = 0
    val_loss = 0   
    
        
    train_y_pred=[]
    train_y_true=[]
    
    val_y_pred=[]
    val_y_true=[]
    
    # Training
    model = model.train()
    for X_batch, y_batch in train_loader:
        model.zero_grad()
        inputs = Variable(X_batch).to(device)
        targets = Variable(y_batch).to(device)
        
        preds = model(inputs)
     
        loss = criterion(preds, targets)
        train_loss += loss.item()
        
        compare_list=[]
        target_list=[]
        
        compare = torch.max(preds, 1)[1]
        
        compare_list.append([i.item() for i in compare])
        compare_list=compare_list[0]
        target_list.append([i.item() for i in targets])
        target_list=target_list[0]
        
        train_correct+=torch.sum(compare==targets).item()
        train_count += X_batch.size(0)
        
        train_y_pred.extend(compare_list)
        train_y_true.extend(target_list)
        
        loss.backward()
        optimizer.step()
    
    train_acc = train_correct/train_count
    avg_train_loss = train_loss/(train_count/batch_size)
 #   print(confusion_matrix(y_pred=train_y_pred,y_true=train_y_true))
    f1=f1_score(y_pred=train_y_pred,y_true=train_y_true)
    
    print('========= Update finished! ===========')
    
    model = model.eval()
    with torch.no_grad():          
        for X_batch, y_batch in test_loader:
            val_count += 1         
            if len(X_batch) < max(filter_list):
                X_batch = torch.LongTensor(X_batch + [word2idx.get("<PAD>")]*(max(filter_sizes)-len(X_batch))).to(device)
            else:
                X_batch = torch.LongTensor(X_batch).to(device)
            inputs = Variable(X_batch).to(device)
            target = Variable(torch.LongTensor(y_batch)).to(device)
            pred = model.predict(inputs, test_batch_size=1)

            loss = criterion(pred, target)
            val_loss += loss.item()
            
            test_pred_list=[]
            test_true_list=[]
            
            _, pred = torch.max(pred, 1)
            true = y_batch.item()
            if true == pred.item():
                val_correct +=1
                          
            test_pred_list.append([i.item() for i in pred])
            test_pred_list=test_pred_list[0]
            test_true_list.append([i.item() for i in target])
            test_true_list=test_true_list[0]
            
            val_y_pred.extend(test_pred_list)
            val_y_true.extend(test_true_list)
            
        val_acc = val_correct/val_count
        avg_val_loss = val_loss/val_count
        
        f1_val=f1_score(y_pred=val_y_pred,y_true=val_y_true)
        
    if epoch % 1 == 0 or epoch == (10-1):
  #      plt.show()
  #      plt.figure(figsize=(10,5))
  #      sns.heatmap(confusion_matrix(y_pred=train_y_pred,y_true=train_y_true),cmap='summer',annot=True)
        
        print('epoch: {:d}'.format(epoch))
        print('train_acc: {:.3f} ({:d}/{:d})'.format(train_acc, train_correct, train_count))
        
        print('train_f1: {:.3f}' .format(f1))
        print('--------------------------')
        
  #      sns.heatmap(confusion_matrix(y_pred=val_y_pred,y_true=val_y_true),cmap='summer',annot=True)
        print('val_acc: {:.3f} ({:d}/{:d})'.format(val_acc, val_correct, val_count))
        print('val_f1: {:.3f}' .format(f1_val))
        print('avg_train_loss: {:.3f}'.format(avg_train_loss))        
        print('avg_val_loss: {:.3f}'.format(avg_val_loss))
        print("==========================================================")
        print('\n')
    

========= Update finished! ===========


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


TypeError: add() received an invalid combination of arguments - got (list), but expected one of:
 * (Tensor other, Number alpha)
 * (Number other, Number alpha)


In [148]:
with torch.no_grad():          
    for X_batch, y_batch in test_loader:
        val_count += 1         
  #      if len(X_batch) < max(filter_list):
   #         X_batch = torch.LongTensor(X_batch.item() + [word2idx.get("<pad>")]*(max(filter_sizes)-len(X_batch))).to(device)
    #    else:
     #       X_batch = torch.LongTensor(X_batch).to(device)
        inputs = Variable(X_batch).to(device)
        target = Variable(torch.LongTensor(y_batch)).to(device)
        pred = model.predict(inputs, test_batch_size=1)

        loss = criterion(pred, target)
        val_loss += loss.item()

        test_pred_list=[]
        test_true_list=[]

        _, pred = torch.max(pred, 1)
        true = y_batch.item()
        if true == pred.item():
            val_correct +=1

        test_pred_list.append([i.item() for i in pred])
        test_pred_list=test_pred_list[0]
        test_true_list.append([i.item() for i in target])
        test_true_list=test_true_list[0]

        val_y_pred.extend(test_pred_list)
        val_y_true.extend(test_true_list)

    val_acc = val_correct/val_count
    avg_val_loss = val_loss/val_count

    f1_val=f1_score(y_pred=val_y_pred,y_true=val_y_true)

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [149]:
print('epoch: {:d}'.format(epoch))
print('train_acc: {:.3f} ({:d}/{:d})'.format(train_acc, train_correct, train_count))

print('train_f1: {:.3f}' .format(f1))
print('--------------------------')

#      sns.heatmap(confusion_matrix(y_pred=val_y_pred,y_true=val_y_true),cmap='summer',annot=True)
print('val_acc: {:.3f} ({:d}/{:d})'.format(val_acc, val_correct, val_count))
print('val_f1: {:.3f}' .format(f1_val))
print('avg_train_loss: {:.3f}'.format(avg_train_loss))        
print('avg_val_loss: {:.3f}'.format(avg_val_loss))
print("==========================================================")
print('\n')

epoch: 0
train_acc: 0.778 (23366/30046)
train_f1: 0.000
--------------------------
val_acc: 0.778 (12038/15480)
val_f1: 0.000
avg_train_loss: 0.536
avg_val_loss: 0.536


